In [10]:
import mujoco

xml_path = "/home/ac/Desktop/2025/project_isaac/trackerLab_private/data/assets/pi_plus_27dof/mjcf/pi+_27dof_0811.xml"
# xml_path = "/home/ac/Desktop/2025/project_isaac/IsaacLab/source/labBundle/factoryIsaac/assets/mujoco_robots/unitree_go2/scene.xml"
model = mujoco.MjModel.from_xml_path(xml_path)
data = mujoco.MjData(model)

In [11]:
for i in range(model.njnt):
    joint_name = model.jnt(i).name
    joint_type = model.jnt_type[i]  
    qpos_addr = model.jnt_qposadr[i]  
    qvel_addr = model.jnt_dofadr[i]  
    print(f"Joint {i}: {joint_name}")
    print(f"  Type: {joint_type} ") # (0=free, 1=hinge, 2=slide, 3=ball)
    print(f"  qpos address: {qpos_addr}")
    print(f"  qvel address: {qvel_addr}")

Joint 0: floating_base_joint
  Type: 0 
  qpos address: 0
  qvel address: 0
Joint 1: waist_joint
  Type: 3 
  qpos address: 7
  qvel address: 6
Joint 2: l_hip_pitch_joint
  Type: 3 
  qpos address: 8
  qvel address: 7
Joint 3: l_hip_roll_joint
  Type: 3 
  qpos address: 9
  qvel address: 8
Joint 4: l_thigh_joint
  Type: 3 
  qpos address: 10
  qvel address: 9
Joint 5: l_calf_joint
  Type: 3 
  qpos address: 11
  qvel address: 10
Joint 6: l_ankle_pitch_joint
  Type: 3 
  qpos address: 12
  qvel address: 11
Joint 7: l_ankle_roll_joint
  Type: 3 
  qpos address: 13
  qvel address: 12
Joint 8: r_hip_pitch_joint
  Type: 3 
  qpos address: 14
  qvel address: 13
Joint 9: r_hip_roll_joint
  Type: 3 
  qpos address: 15
  qvel address: 14
Joint 10: r_thigh_joint
  Type: 3 
  qpos address: 16
  qvel address: 15
Joint 11: r_calf_joint
  Type: 3 
  qpos address: 17
  qvel address: 16
Joint 12: r_ankle_pitch_joint
  Type: 3 
  qpos address: 18
  qvel address: 17
Joint 13: r_ankle_roll_joint
  Type: 

In [ ]:
print(data.ctrl)
for i in range(model.nu):
    actuator_name = model.actuator(i).name
    joint_id = model.actuator_trnid[i, 0]  # 执行器控制的关节ID
    joint_name = model.joint(joint_id).name if joint_id >= 0 else "None"
    
    # 打印执行器信息
    print(f"Actuator {i}: '{actuator_name}'")
    print(f"  Controls joint: '{joint_name}' (ID: {joint_id})")
    print(f"  ctrl index: {i} (current value: {data.ctrl[i]})")
    print(f"  Actuator type: {model.actuator_trntype[i]}")  # 执行器传输类型

In [ ]:
import numpy as np
from sim2simlib.model.sim2sim_base import Sim2Sim_Config, Sim2Sim_Base_Model, Observations_Config, Actions_Config, DC_Motor_Cfg

config = Sim2Sim_Config(
    robot_name='Go2',
    simulation_dt=0.005,
    control_decimation=4,
    xml_path="/home/ac/Desktop/2025/project_isaac/IsaacLab/source/mujoco_menagerie/unitree_go2/scene.xml",
    policy_path="/home/ac/Desktop/2025/project_isaac/IsaacLab/source/labBundle/factoryIsaac/logs/exp1/Go2_doublehand_flat_action_nonoise/2025-08-06_13-55-54_a0.15_seed423/exported/policy.pt",
    policy_joint_names=[ 
            "FL_hip_joint", "FL_thigh_joint", "FL_calf_joint",
            "FR_hip_joint", "FR_thigh_joint", "FR_calf_joint",
            "RL_hip_joint", "RL_thigh_joint", "RL_calf_joint",
            "RR_hip_joint", "RR_thigh_joint", "RR_calf_joint",   
        ],
    observation_cfg=Observations_Config(
        base_observations_terms=['ang_vel', 
                             'cmd', 
                             'gravity_orientation', 
                             'joint_pos', 
                             'joint_vel',
                             'action'],
        scale={
                'lin_vel': 2.0,
                'ang_vel': 0.25,
                'cmd': 1.0,
                'gravity_orientation': 1.0,
                'joint_pos': 1.0,
                'joint_vel': 0.05
            },
        ),
    action_cfg=Actions_Config(
        action_clip=(-100.0, 100.0),
        scale=0.25
    ),
    motor_cfg=DC_Motor_Cfg(
        effort_limit=23.5,
        saturation_effort=23.5,
        velocity_limit=30.0,
        stiffness=25.0,
        damping=0.5
    ),

    default_pos=np.array([0.0, 0.0, 0.27], dtype=np.float32),
    default_angles=np.array([ 0.0, 0.9, -1.8, 0.0, 0.9, -1.8, 0.0, 0.9, -1.8, 0.0, 0.9, -1.8 ], dtype=np.float32),
)

In [2]:
mujoco_model = Sim2Sim_Base_Model(config)

[INFO] MuJoCo joint names: ['', 'FL_hip_joint', 'FL_thigh_joint', 'FL_calf_joint', 'FR_hip_joint', 'FR_thigh_joint', 'FR_calf_joint', 'RL_hip_joint', 'RL_thigh_joint', 'RL_calf_joint', 'RR_hip_joint', 'RR_thigh_joint', 'RR_calf_joint']
[INFO] Policy joint names: ['FL_hip_joint', 'FL_thigh_joint', 'FL_calf_joint', 'FR_hip_joint', 'FR_thigh_joint', 'FR_calf_joint', 'RL_hip_joint', 'RL_thigh_joint', 'RL_calf_joint', 'RR_hip_joint', 'RR_thigh_joint', 'RR_calf_joint']
